In [1]:
#%pip install transformers
#%pip install torch
#%pip install datasets

## Readme
Uncomment code below, to install requirements, then restart the kernel, comment out cell below and run all cells. 

In [2]:
#%pip install -r requirements.txt

In [3]:
import torch

device = "cpu"
# setup optimal acceleration device 
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal Performance Shaders on macOS
elif torch.cuda.is_available():
    device = torch.device("cuda")  # to check if cuda is an option https://www.restack.io/p/gpu-computing-answer-is-my-gpu-cuda-enabled-cat-ai

print(f"device is : {device}")

device is : cuda


In [4]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

#finetuning on IMDb
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

model_checkpoint = 'distilbert/distilbert-base-uncased'

# Replace AutoModelForMaskedLM with the correct class for your task, e.g., AutoModelForSequenceClassification
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)



d:\UT-Folder\Masters\M6\AP info\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
imdb_dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [6]:
sample = imdb_dataset["unsupervised"].shuffle(seed=97).select(range(3))

In [7]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

Map: 100%|██████████| 50000/50000 [00:09<00:00, 5407.72 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [8]:
chunk_size = tokenizer.model_max_length // 2
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) if isinstance(examples[k][0], list) else examples[k] for k in examples.keys()}
    # Compute the total length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new "labels" column that is a copy of "input_ids"
    result["labels"] = result["input_ids"].copy()
    return result


In [9]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map: 100%|██████████| 50000/50000 [01:12<00:00, 690.17 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30639
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 29946
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61465
    })
})

In [10]:
#deviate from the tutoral at this point, and look at documention instead 

In [11]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.pad_token = "[PAD]" #tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="movie_model",
    save_strategy="epoch",
    eval_strategy="epoch", 
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
    per_device_train_batch_size=32,  # lower this if memory error
)

trainer = Trainer(
    model=model.to(device),  # Move model to the specified device
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()


d:\UT-Folder\Masters\M6\AP info\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\windows-game-pc\AppData\Local\Temp\ipykernel_20988\1787854875.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 52%|█████▏    | 500/958 [05:20<04:54,  1.55it/s]

{'loss': 2.5085, 'grad_norm': 3.7063238620758057, 'learning_rate': 9.561586638830899e-06, 'epoch': 0.52}


                                                 
100%|██████████| 958/958 [17:56<00:00,  1.90it/s]  

{'eval_loss': 2.3098809719085693, 'eval_runtime': 464.6123, 'eval_samples_per_second': 64.454, 'eval_steps_per_second': 8.058, 'epoch': 1.0}


100%|██████████| 958/958 [17:57<00:00,  1.12s/it]

{'train_runtime': 1077.4063, 'train_samples_per_second': 28.438, 'train_steps_per_second': 0.889, 'train_loss': 2.4723631793123695, 'epoch': 1.0}


TrainOutput(global_step=958, training_loss=2.4723631793123695, metrics={'train_runtime': 1077.4063, 'train_samples_per_second': 28.438, 'train_steps_per_second': 0.889, 'total_flos': 2030770630812672.0, 'train_loss': 2.4723631793123695, 'epoch': 1.0})

In [13]:
output_dir = "finetuned_models"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('finetuned_models\\tokenizer_config.json',
 'finetuned_models\\special_tokens_map.json',
 'finetuned_models\\vocab.txt',
 'finetuned_models\\added_tokens.json',
 'finetuned_models\\tokenizer.json')

In [ ]:
from transformers import pipeline

mask_filler_base = pipeline(
    "fill-mask", model=model_checkpoint, device=device
)

mask_filler_finetuned = pipeline(
    "fill-mask", model="finetuned_models/", device=device
)

RuntimeError: PyTorch is not linked with support for mps devices

In [ ]:
text = "This is a great [MASK]"

#origional model
for pred in mask_filler_base(text):
    print(f"Origional >>> {pred['sequence']}")

print("\n")

#new IMDb finetuned model 
for pred in mask_filler_finetuned(text):
    print(f"Updated model >>> {pred['sequence']}")
